In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import google.generativeai as genai

: 

In [ ]:
# --- Configure Gemini API ---
genai.configure(api_key="YOUR_GEMINI_API_KEY")
model = genai.GenerativeModel("gemini-pro")

# --- Load Datasets ---
df_health1 = pd.read_csv("health_dataset_1.csv")
df_health2 = pd.read_csv("health_dataset_2.csv")

# --- Data Preprocessing ---
# Handle missing values
imputer = SimpleImputer(strategy='mean')
df_health1.iloc[:, 1:] = imputer.fit_transform(df_health1.iloc[:, 1:])
df_health2.iloc[:, 2:] = imputer.fit_transform(df_health2.iloc[:, 2:])

# --- Join Data Temporarily ---
def join_on_the_fly(patient_number):
    df1_row = df_health1[df_health1['Patient_Number'] == patient_number]
    df2_rows = df_health2[df_health2['Patient_Number'] == patient_number]
    if df1_row.empty or df2_rows.empty:
        return None
    
    # Take mean of physical activity over 10 days
    steps_mean = df2_rows['Physical_activity'].mean()
    df1_row = df1_row.copy()
    df1_row['Avg_Physical_Activity'] = steps_mean
    return df1_row

# --- Generate Insight from Gemini ---
def generate_insight_from_patient_data(patient_data_df, user_query):
    if patient_data_df is None or user_query.strip() == "":
        return "No matching patient data or empty query."
    
    input_text = f"""
    Given the following patient health data:

    {patient_data_df.to_string(index=False)}

    Answer this query: {user_query}
    """

    response = model.generate_content(input_text)
    return response.text

# --- Streamlit UI ---
st.title("GenAI Health Advisor - Gemini Edition")
st.write("Enter a patient number to retrieve and analyze their health data.")

patient_number_input = st.number_input("Patient Number", min_value=1, max_value=20000, step=1)
user_query = st.text_input("Enter your query about the patient's health:", "Suggest health improvement tips")

if st.button("Get Health Insights"):
    patient_data = join_on_the_fly(patient_number_input)
    if patient_data is not None:
        st.dataframe(patient_data)
        insight = generate_insight_from_patient_data(patient_data, user_query)
        st.write("\n### Generated Insight:")
        st.write(insight)
    else:
        st.error("Patient not found in one of the datasets.")